Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.


In [ ]:
import numpy as np
import pandas as pd
import math
import random

In [ ]:
#wspinaczka z multistartem
distance = np.array([[0,2,4,1,5],
                    [2,0,3,8,7],
                    [4,3,0,5,1],
                    [1,8,5,0,3],
                    [5,7,1,3,0]])

In [ ]:
def Odleglosc(distance, trasa):
    suma = 0
    n = len(trasa)
    
    for i, current_city in enumerate(trasa):
        next_city = trasa[(i + 1) % n]  # Use modulo to handle the last element
        suma += distance[current_city][next_city]

    return suma

In [25]:
#swap
def Swap(trasa):
    trasa_temp = trasa.copy()
    #print(trasa)
    swap = np.random.choice(trasa_temp,2,replace=False)
    temp = trasa_temp[swap[0]]
    trasa_temp[swap[0]] = trasa_temp[swap[1]]
    trasa_temp[swap[1]] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

def Swap_tabu(trasa, miasto1, miasto2):
    trasa_temp = trasa.copy()
    #print(trasa)
    #swap = np.random.choice(trasa_temp,1)
    temp = trasa_temp[miasto1]
    trasa_temp[miasto1] = trasa_temp[miasto2]
    trasa_temp[miasto2] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

#insercja
def Insercja(trasa):
    trasa_temp = trasa.copy()
    insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[insercja[0]]
    trasa_temp = np.delete(trasa_temp,insercja[0])
    trasa_temp = np.insert(trasa_temp,insercja[1],temp)
    #print(trasa_temp)
    return trasa_temp

def Insercja_tabu(trasa, miasto1, miasto2):
    trasa_temp = trasa.copy()
    #insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[miasto1]
    trasa_temp = np.delete(trasa_temp,miasto1)
    trasa_temp = np.insert(trasa_temp,miasto2,temp)
    #print(trasa_temp)
    return trasa_temp    

def Zamiana(trasa):
    trasa_temp = trasa.copy()
    zamiana = np.random.choice(trasa_temp,2,replace=False)
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

def Zamiana_tabu(trasa, miasto1, miasto2):
    trasa_temp = trasa.copy()
    zamiana = [miasto1, miasto2]
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

In [ ]:
#Zamiana(trasa)

In [ ]:
#trasa = np.arange(len(distance))
#np.random.shuffle(trasa)
#Insercja(trasa)

In [ ]:
#temp = trasa[0] #temp = 4
#trasa = np.delete(trasa,0)
#trasa = np.insert(trasa,4,3)

In [ ]:
#ruch polegający na swapie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_swap(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Swap(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl, trasa_new, odl_new

#ruch polegający na insercji, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_insercja(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Insercja(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl

#ruch polegający na zamianie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_zamiana(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Zamiana(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new  
    return trasa, odl

In [19]:
distance = pd.read_excel("Dane_TSP_48.xlsx")
distance = distance.to_numpy()
distance = np.delete(distance,0,1)
distance

array([[   0, 1495,  381, ...,  488,  753, 1184],
       [1495,    0, 1135, ..., 1727, 1388,  661],
       [ 381, 1135,    0, ...,  640,  591,  810],
       ...,
       [ 488, 1727,  640, ...,    0,  520, 1242],
       [ 753, 1388,  591, ...,  520,    0,  801],
       [1184,  661,  810, ..., 1242,  801,    0]], dtype=int64)

In [ ]:
def IHC(distance, ruch, n_iter_ruch, n_iter):  # ruch = {Swap, Insercja, Zamiana}
    min_iter = []
    for i in range(n_iter):
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                if odl_new < odl:
                    odl = odl_new
                    trasa = trasa_new
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = IHC(distance,Swap,1000,100)
wynik.sort(key=lambda x: x[2])
wynik[0:10]



In [ ]:
def Geometric(T,alpha):
    return alpha*T

def Slow(T,alpha):
    return T/(1+alpha*T)


In [ ]:
def SA(distance, ruch, n_iter_ruch, n_iter, T_start, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,1000,100,100,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

In [ ]:
# wersja SA gdzie warunkiem stopu jest temperatura a nie liczba iteracji 
# (chyba lepiej tak bo upewniamy sie ze temperatura jest niska a podajac liczbe iteracji mozemy skonczyc na wysokiej temperaturze)
# mozna tez dodac warunek stopu ze po np 100 iteracjach bez poprawy rozwiazania to koniec

def SA(distance, ruch, n_iter, T_start, T_stop, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        while(T > T_stop):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,100,100,0.01,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

In [ ]:
# funkcje pomocnicze do sprawdzenia ile iteracji odpowiada parametrowi T_stop
def T_stopToIter(T_start, T_stop, cooling, alpha):
    T = T_start
    i = 0
    while(T > T_stop):
        T = cooling(T,alpha)
        i += 1
    return i
def IterToT_stop(T_start, n_iter, cooling, alpha):
    T = T_start
    for i in range(n_iter):
        T = cooling(T,alpha)
    return T
print(T_stopToIter(100,0.01,Slow,0.1))
IterToT_stop(100,100,Slow,0.1)

algorytm genetyczny

In [ ]:
def ox(r1, r2):
    n = len(r1)
    a = random.randint(0,n-1)
    b = random.randint(0,n-1)
    if a > b:
        a,b = b,a
    p1 = r1.copy(); p2 = r2.copy()
    p1[a:b] = r2[a:b]; p2[a:b] = r1[a:b]
    return p1, p2

In [ ]:
r1 = [1,2,3,4,5,6,7,8,9]
r2 = [4,5,2,1,8,7,6,9,3]
n = len(r1)
a = 3
b = 7
section = np.arange(a)
section = np.append(section,np.arange(b,n))
if a > b:
    a,b = b,a
p1 = r1.copy()
p2 = r2.copy()
p1[0:a], p1[b:n] = np.zeros(a).astype(int), np.zeros(n-b).astype(int)
p2[0:a], p2[b:n] = np.zeros(a).astype(int), np.zeros(n-b).astype(int)
print(p1,p2)

j = 0
for i in section:
    while True:
        if r2[i] not in p1:
            p2[i] = r1[j]
            j+=1
            break
        else:
            j+=1
print(p1,p2)

In [ ]:
# #algorytm ze swapem
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_swap(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))


In [ ]:
# #algorytm z insercją
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_insercja(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [ ]:
# #algorytm z zamianą
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 1000

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_zamiana(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [ ]:
#NN
min_odl = []
for n in range(len(distance)):
    potencjalne = np.arange(len(distance))
    trasa = np.zeros(len(distance),dtype=int)
    trasa[0] = potencjalne[n]
    potencjalne = np.delete(potencjalne,int(trasa[0]))

    for cur in range(1,len(trasa)):
        odleglosci = []
        for i in range(len(potencjalne)):
            trasa[cur] = potencjalne[i]
            odleglosci.append((potencjalne[i],Odleglosc(distance,trasa[0:cur+1])))
        odleglosci.sort(key=lambda x: x[1])
        trasa[cur] = odleglosci[0][0]
        potencjalne = np.delete(potencjalne,potencjalne==trasa[cur])
    min_odl.append((Odleglosc(distance,trasa),trasa))
    
min_odl.sort(key=lambda x:x[0])
min_odl[:10]

In [32]:
#lista tabu, tabu search
#dlugosc listy tabu jako parametr
def TS(distance, ruch, tabu_len, n_iter_ruch, n_iter):  # ruch = {Swap_tabu, Insercja_tabu, Zamiana_tabu}
    min_iter = []
    for i in range(n_iter):
        #losujemy trasę 
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        #inicjalizujemy listę tabu
        lista_tabu = []
        #rozpoczynamy pętlę która odpowiada za powtarzanie każdego ruchu
        for k in range(n_iter_ruch):
            #inicjalizujemy listę potencjalnych zamian
            potencjalne_zamiany = []
            #losujemy jedno stałe miasto w danym ruchu
            miasto = np.random.choice(trasa,1)
            #rozpoczynamy pętle która sprawdzi odległość dla ruchu pomiędzy wylosowanym stałym 
            #miastem a wszystkimi pozostałymi
            for j in range(len(distance)):
                trasa_temp = trasa.copy()
                #sprawdzamy czy kombinacja miast znajduje się na liście tabu
                if [miasto, j] not in lista_tabu and [j,miasto] not in lista_tabu:
                    #jeśli nie znajduje się na liście tabu to tworzymy nową trasę
                    trasa_temp = ruch(trasa_temp,miasto,j)
                    #do potencjalnych zamian dodajemy nową odległość oraz pomiędzy jakimi miastami wykonano ruch
                    potencjalne_zamiany.append([Odleglosc(distance,trasa_temp),miasto,j])
            #sortujemy wszystkie potencjalne ruchy według odległości
            potencjalne_zamiany.sort(key=lambda x:x[0])
            #wybieramy najlepszy możliwy ruch i wykonujemy go na pierwotnej trasie
            trasa = ruch(trasa,potencjalne_zamiany[0][1],potencjalne_zamiany[0][2])
            #dodajemy ruch do listy tabu
            lista_tabu.append([potencjalne_zamiany[0][1],potencjalne_zamiany[0][2]])
            #lista tabu zapełnia się z każdym ruchem, jeśli dojdzie do maksymalnej długości to usuwamy
            #najwcześniejszą wartość z listy
            if len(lista_tabu) == tabu_len:
                lista_tabu.remove(lista_tabu[0])
        #uzupełniamy listę potencjalnych tras
        min_iter.append([i,trasa,Odleglosc(distance, trasa)])
    return min_iter
wynik = TS(distance,Swap_tabu,5,1000,100)
wynik.sort(key=lambda x: x[2])
wynik[0:10]
#zamiana chyba bez sensu

[[54,
  array([15,  0,  7, 37, 39, 14, 10, 22, 13, 33,  2, 21,  8, 30, 43, 17,  6,
         27, 42, 16, 26, 18, 36,  5, 29, 35, 45, 32, 11, 19, 46, 20, 31, 38,
         12, 24, 47,  4, 41, 23,  9, 44, 34,  3, 25,  1, 28, 40]),
  11937],
 [30,
  array([35,  6, 17, 43, 14, 11, 32, 45, 30, 37,  8, 39, 10, 12, 22,  2, 21,
          0,  7, 15, 40, 33,  4, 47, 38, 31, 23, 41,  9, 44, 34,  3, 25,  1,
         28, 13, 24, 20, 46, 19, 29, 42, 16, 26, 18, 36,  5, 27]),
  12041],
 [47,
  array([26, 18, 36,  5, 27, 45, 32, 11, 10, 22, 12, 13, 24,  4, 47, 23,  9,
         44, 34,  3, 25, 41,  1, 28, 38, 31, 20, 46, 19, 35,  6, 17, 43, 30,
         37,  7, 15, 40, 33,  2, 21,  0,  8, 39, 14, 29, 42, 16]),
  12197],
 [24,
  array([11, 10, 13, 24, 38, 31, 47,  4, 41, 23,  9, 44, 34,  3, 25,  1, 28,
         33,  2, 45, 32, 14, 39,  8, 21, 40, 15,  0,  7, 37, 30, 43, 17,  6,
         27, 35, 36, 18, 26, 16, 42,  5, 29, 19, 46, 20, 12, 22]),
  12314],
 [60,
  array([ 9, 44, 34,  3, 25,  1, 28,  4, 47, 4